In [4]:
import time
import pandas as pd
from groupmembercrawl import FacebookGroupMemberCrawl
from grouppostcrawl import FacebookGroupPostCrawl


In [5]:
username = 'phanmthihoa05@gmail.com'
password = 'Hung26082003'
group_id = '811896080494851'
scroll_count = 5
try:
    scraper = FacebookGroupPostCrawl(username= username, password = password, group_id = group_id, scroll_count= scroll_count)
    if scraper.login():
        print('-----------------')
        postlist = scraper.get_group_posts()
        scraper.save_post_to_excel(postlist)
        deitalpost = scraper.get_detail_each_post(postlist)
        scraper.save_reactions_to_excel(deitalpost)
        time.sleep(10)
except Exception as e:
    pass


====== Facebook Group post Scraper ======
Login success
-----------------
Found 5 post elements on scroll 1
Found 5 post elements on scroll 2
Found 8 post elements on scroll 3
Found 8 post elements on scroll 4
Found 10 post elements on scroll 5
Post data saved to ex_post.xlsx
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='xrbpyxo x6ikm8r x10wlt62 xlyipyv x1exxlbk']
Found element with XPath: //span[@class='x

In [6]:
# def save_to_excel(members):
#             file_name = f"ex_post.xlsx"
#             df = pd.DataFrame(members, columns=["Post ID", "type"])
#             df.to_excel(file_name, index=False)
#             print(f"Data saved to {file_name}")
# save_to_excel(all_post)